In [28]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf


In [29]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Physical devices cannot be modified after being initialized


In [30]:
X_train , X_test , Y_train , Y_test = np.load(
    './crawling_data/news_data_max_29_wordsize_12831.npy', allow_pickle=True)
print(X_train.shape , Y_train.shape)
print(X_test.shape , Y_test.shape)

(12537, 29) (12537, 6)
(1394, 29) (1394, 6)


In [31]:
model = Sequential()
model.add(Embedding(13000, 150 , input_length=29))  #데이터 개수 , 차원개수 200~300(단어학습기준)
model.add(Conv1D(32,kernel_size=5 , padding='same' , activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(LSTM(128, activation='tanh', return_sequences=True ))
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh', return_sequences=True ))
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh', return_sequences=True ))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 150)           1950000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 29, 32)            24032     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 29, 32)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 29, 128)           82432     
_________________________________________________________________
dropout_6 (Dropout)          (None, 29, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 29, 64)            49408     
_________________________________________________________________
dropout_7 (Dropout)          (None, 29, 64)           

#데이터 개수 , 차원이 늘어 날수록 데이터개수 많아야 한다 (조밀도 하락) 차원이 많아서 임의적으로 줄임
# conv1D 빠르다, dense layer 감소하게 해줌


In [32]:
model.compile(loss='categorical_crossentropy' , optimizer='adam',metrics=['accuracy'])

In [33]:
fit_hist = model.fit(X_train, Y_train, epochs=10, verbose=1, validation_data=(X_test, Y_test))
model.save('./output/news_category_classfications_model_{}.h5'.format(fit_hist.history['val_accuracy'][-1]))

Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_2/conv1d_2/conv1d (defined at Users\SteveLee\AppData\Local\Temp\ipykernel_16216\2877729266.py:1) ]]
	 [[gradient_tape/sequential_2/embedding_2/embedding_lookup/Reshape/_32]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_2/conv1d_2/conv1d (defined at Users\SteveLee\AppData\Local\Temp\ipykernel_16216\2877729266.py:1) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_25395]

Function call stack:
train_function -> train_function


In [ ]:
plt.plot(fit_hist.history['accuracy'],label = 'accuracy')
plt.plot(fit_hist.history['val_accuracy'], label = 'val_accuracy')
plt.legend()
plt.show()